In [1]:
import semantic_kernel as sk
import os
import logging
from dotenv import load_dotenv, find_dotenv
from helpers.LoggingHelper import LoggingHelper

_ = load_dotenv(find_dotenv()) # read local .env file
print(os.getenv("OPENAI_API_BASE"))

lh = LoggingHelper()
logger = lh.get_logger("semantic-kernel", logging.WARN)


https://alkopenai2.openai.azure.com/


In [2]:

from semantic_kernel.connectors.ai.open_ai import (
    AzureChatCompletion,
    AzureTextCompletion,
)

kernel = sk.Kernel(log=logger)
kernel.add_chat_service(
    "chat_completion",
    AzureChatCompletion(
        "GPT35",
        os.getenv("OPENAI_API_BASE"),
        os.getenv("OPENAI_API_KEY")
    ),
)

In [3]:
# Now we need to import the plugin
from plugins.JarvisPlugin.Tasks import Tasks

# Now you can import the plugin importing skill directly from the function you declared
# in the plugin directory. The import_skill does not need the path, it only need an
# instance of the skill and the name of the skill
tasks_plugin = kernel.import_skill(Tasks(), skill_name="JarvisPlugin")


In [4]:
from pprint import pprint
# want to print all the keys of extractaudio_plugin that is a dictionary

pprint (tasks_plugin["ChangeTaskTitle"])

In [5]:
# We start using a planner, that allows us not to have to manually call functions
from semantic_kernel.planning.basic_planner import BasicPlanner
planner = BasicPlanner()

In [6]:
# now we can use the planner
question = "I want to change title of Task_34 to 'Hello semantic kernel'"
plan = await planner.create_plan_async(question, kernel)


In [7]:
pprint(plan)


Prompt: 
You are a planner for the Semantic Kernel.
Your job is to create a properly formatted JSON plan step by step, to satisfy the goal given.
Create a list of subtasks based off the [GOAL] provided.
Each subtask must be from within the [AVAILABLE FUNCTIONS] list. Do not use any functions that are not in the list.
Base your decisions on which functions to use from the description and the name of the function.
Sometimes, a function may take arguments. Provide them if necessary.
The plan should be as short as possible.
For example:

[AVAILABLE FUNCTIONS]
EmailConnector.LookupContactEmail
description: looks up the a contact and retrieves their email address
args:
- name: the name to look up

WriterSkill.EmailTo
description: email the input text to a recipient
args:
- input: the text to email
- recipient: the recipient's email address. Multiple addresses may be included if separated by ';'.

WriterSkill.Translate
description: translate the input to another language
args:
- input: the te

In [8]:

result = await planner.execute_plan_async(plan, kernel)
print(f"Plan results: {result}")


Now call the api to change the title of the task Task_34 to Hello semantic kernel
Plan results: 7


In [9]:

print(plan.generated_plan)

{
    "input": "Change Task Title",
    "subtasks": [
        {"function": "JarvisPlugin.ChangeTaskTitle", "args": {"new_title": "Hello semantic kernel", "task_id": "Task_34"}}
    ]
}


In [10]:

print(f"Plan results: {result}")

Plan results: 7
------------------
{
    "input": "Change Task Title",
    "subtasks": [
        {"function": "JarvisPlugin.ChangeTaskTitle", "args": {"new_title": "Hello semantic kernel", "task_id": "Task_34"}}
    ]
}


In [ ]:
# now we can create a stepwise planner
from semantic_kernel.planning.stepwise_planner import StepwisePlanner
sw_planner = StepwisePlanner(kernel)


In [32]:

from pprint import pprint
question = "I want to change title of Task_34 to 'Hello semantic kernel'"
plan = sw_planner.create_plan(question)

2023-11-30 18:59:14,624 - semantic-kernel - DEBUG - Rendering string template: [INSTRUCTION]
Answer the following questions as accurately as possible using the provided functions.

[AVAILABLE FUNCTIONS]
The function definitions below are in the following format:
<functionName>: <description>
  inputs:
    - <parameterName>: <parameterDescription>
    - ...

{{$function_descriptions}}
[END AVAILABLE FUNCTIONS]

[USAGE INSTRUCTIONS]
To use the functions, specify a JSON blob representing an action. The JSON blob should contain an "action" key with the name of the function to use, and an "action_variables" key with a JSON object of string values to use when calling the function.
Do not call functions directly; they must be invoked through an action.
The "action_variables" value should always include an "input" key, even if the input value is empty. Additional keys in the "action_variables" value should match the defined [PARAMETERS] of the named "action" in [AVAILABLE FUNCTIONS].
Dictionar

Now call the api to change the title of the task Task_34 to Hello semantic kernel


2023-11-30 18:59:20,153 - semantic-kernel - DEBUG - Response: [FINAL ANSWER]
To change the title of Task_34 to 'Hello semantic kernel', we can use the JarvisPlugin.ChangeTaskTitle function with the following action:
{"action": "JarvisPlugin.ChangeTaskTitle", "action_variables": {"new_title": "Hello semantic kernel", "task_id": "Task_34"}}
The function will return a value of 7.
2023-11-30 18:59:20,155 - semantic-kernel - DEBUG - Final Answer: To change the title of Task_34 to 'Hello semantic kernel', we can use the JarvisPlugin.ChangeTaskTitle function with the following action:
{"action": "JarvisPlugin.ChangeTaskTitle", "action_variables": {"new_title": "Hello semantic kernel", "task_id": "Task_34"}}
The function will return a value of 7.
2023-11-30 18:59:20,155 - semantic-kernel - DEBUG - Scratchpad: This was my previous work (but they haven't seen any of it! They only see what I return as final answer):
[THOUGHT]
We can use the JarvisPlugin.ChangeTaskTitle function to change the titl

Plan results: To change the title of Task_34 to 'Hello semantic kernel', we can use the JarvisPlugin.ChangeTaskTitle function with the following action:
{"action": "JarvisPlugin.ChangeTaskTitle", "action_variables": {"new_title": "Hello semantic kernel", "task_id": "Task_34"}}
The function will return a value of 7.
------------------
SKContext(memory=<semantic_kernel.memory.null_memory.NullMemory object at 0x1269bf990>, variables=ContextVariables(variables={'input': 'To change the title of Task_34 to \'Hello semantic kernel\', we can use the JarvisPlugin.ChangeTaskTitle function with the following action:\n{"action": "JarvisPlugin.ChangeTaskTitle", "action_variables": {"new_title": "Hello semantic kernel", "task_id": "Task_34"}}\nThe function will return a value of 7.', 'agent_scratch_pad': 'This was my previous work (but they haven\'t seen any of it! They only see what I return as final answer):\n[THOUGHT]\nWe can use the JarvisPlugin.ChangeTaskTitle function to change the title of th

In [ ]:

pprint(plan)


In [ ]:

result = await plan.invoke_async()


In [ ]:

print(f"Plan results: {result}")

print("------------------")
pprint(result)
